In [24]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [27]:
out='../output/'
inp='../input/monthly/'
pref='dam'

In [28]:
dat=pd.DataFrame(columns=[0,1,2])
onlyfiles = [f for f in listdir(inp) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat

,0,1,2
0,Date:Time,Price(USD/MWh),Turnover(MWh)
1,(2016/01/01) 00-01,40.1,477.5
2,(2016/01/01) 01-02,40.08,486.6
3,(2016/01/01) 02-03,40.06,495.6
4,(2016/01/01) 03-04,40.08,496.6
...,...,...,...
720,(2021/09/30) 23-24,51.24,193
721,Min,32.04,5
722,Max,304.75,325
723,Average,99.46420833333333333333333333,159.05666666666666666666666667


In [29]:
dat.drop(dat[dat[0] == 'Date:Time'].index, inplace = True)
dat.drop(dat[dat[0] == 'Min'].index, inplace = True)
dat.drop(dat[dat[0] == 'Max'].index, inplace = True)
dat.drop(dat[dat[0] == 'Average'].index, inplace = True)
dat.drop(dat[dat[0] == 'Total'].index, inplace = True)
dat.columns=['datehour','price','volume']
dat['price']=dat['price'].astype(float)
dat['volume']=dat['volume'].astype(float)
dat

,datehour,price,volume
1,(2016/01/01) 00-01,40.10,477.5
2,(2016/01/01) 01-02,40.08,486.6
3,(2016/01/01) 02-03,40.06,495.6
4,(2016/01/01) 03-04,40.08,496.6
5,(2016/01/01) 04-05,40.07,502.9
...,...,...,...
716,(2021/09/30) 19-20,300.08,42.0
717,(2021/09/30) 20-21,253.30,128.0
718,(2021/09/30) 21-22,77.08,161.0
719,(2021/09/30) 22-23,54.27,182.7


In [30]:
dat['date']=pd.to_datetime(dat.datehour.str.slice(1,11),format="%Y/%m/%d")
dat['hour']=dat.datehour.str.slice(13,15).astype(int)
dat

,datehour,price,volume,date,hour
1,(2016/01/01) 00-01,40.10,477.5,2016-01-01,0
2,(2016/01/01) 01-02,40.08,486.6,2016-01-01,1
3,(2016/01/01) 02-03,40.06,495.6,2016-01-01,2
4,(2016/01/01) 03-04,40.08,496.6,2016-01-01,3
5,(2016/01/01) 04-05,40.07,502.9,2016-01-01,4
...,...,...,...,...,...
716,(2021/09/30) 19-20,300.08,42.0,2021-09-30,19
717,(2021/09/30) 20-21,253.30,128.0,2021-09-30,20
718,(2021/09/30) 21-22,77.08,161.0,2021-09-30,21
719,(2021/09/30) 22-23,54.27,182.7,2021-09-30,22


In [31]:
dat=dat.drop(['datehour'],1)
dat

,price,volume,date,hour
1,40.10,477.5,2016-01-01,0
2,40.08,486.6,2016-01-01,1
3,40.06,495.6,2016-01-01,2
4,40.08,496.6,2016-01-01,3
5,40.07,502.9,2016-01-01,4
...,...,...,...,...
716,300.08,42.0,2021-09-30,19
717,253.30,128.0,2021-09-30,20
718,77.08,161.0,2021-09-30,21
719,54.27,182.7,2021-09-30,22


In [32]:
hourly=dat.set_index(pd.DatetimeIndex(dat['date']))
hourly['year']=hourly.index.year
hourly['month']=hourly.index.month
hourly['day']=hourly.index.day
hourly['day_of_week']=hourly.index.dayofweek
hourly=hourly.drop(['date'],axis=1)
hourly

,price,volume,hour,year,month,day,day_of_week
date,,,,,,,
2016-01-01,40.10,477.5,0,2016,1,1,4
2016-01-01,40.08,486.6,1,2016,1,1,4
2016-01-01,40.06,495.6,2,2016,1,1,4
2016-01-01,40.08,496.6,3,2016,1,1,4
2016-01-01,40.07,502.9,4,2016,1,1,4
...,...,...,...,...,...,...,...
2021-09-30,300.08,42.0,19,2021,9,30,3
2021-09-30,253.30,128.0,20,2021,9,30,3
2021-09-30,77.08,161.0,21,2021,9,30,3


In [33]:
hourly['value']=hourly['price']*hourly['volume']
daily=hourly.reset_index().groupby(['year','month','day']).sum().drop(['hour','day_of_week'],axis=1)
daily['price_weight_av']=daily['value']/daily['volume']
daily=daily.drop(['price'], axis=1)
daily['price_max']=hourly[['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_min']=hourly[['year','month','day','price']].groupby(['year','month','day']).min()
daily['price_mean']=hourly[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['power_max']=hourly[['year','month','day','volume']].groupby(['year','month','day']).max()
daily['power_min']=hourly[['year','month','day','volume']].groupby(['year','month','day']).min()
daily['power_mean']=hourly[['year','month','day','volume']].groupby(['year','month','day']).mean()

daily['price_morning_max']=hourly.loc[hourly['hour']<11][['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_evening_max']=hourly.loc[hourly['hour']>=11][['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_standard']=hourly.loc[(hourly['hour']>=12) & (hourly['hour']<=15)][['year','month','day','price']].groupby(['year','month','day']).mean()
daily=daily.reset_index()
daily['date']=pd.to_datetime(daily.reset_index()[['year', 'month', 'day']])
daily=daily.set_index(['year','month','day'])
daily


volume       value  price_weight_av  price_max  price_min  \
year month day                                                              
2016 1     1    7873.8  336882.508        42.785251      60.08      35.09   
           2    7319.5  345982.021        47.268532     118.49      40.03   
           3    7198.7  351712.733        48.857812     118.47      44.06   
           4    4457.0  225938.810        50.693024     177.66      46.70   
           5    3750.4  179068.882        47.746609     181.40      42.06   
...                ...         ...              ...        ...        ...   
2021 9     26   3809.0  141505.170        37.150215      57.26      32.06   
           27   3148.6  320320.984       101.734417     304.75      55.06   
           28   3059.4  299921.978        98.032940     303.65      50.09   
           29   2496.3  280136.224       112.220576     304.25      53.13   
           30   2534.8  289501.568       114.210813     300.09      50.09   

                price_mean  power_max  power_min  power_mean  \
year month day                                                 
2016 1     1     45.982500      502.9       53.0  328.075000   
           2     54.114167      486.0       40.0  304.979167   
           3     54.959167      486.0       40.0  299.945833   
           4     80.164583      486.0        0.0  185.708333   
           5     84.503333      436.0        0.0  156.266667   
...                    ...        ...        ...         ...   
2021 9     26    38.151250      212.0       56.0  158.708333   
           27   122.924167      216.5        5.0  131.191667   
           28   121.011667      190.5       27.0  127.475000   
           29   132.382000      189.4       15.0  124.815000   
           30   130.391500      193.0       41.1  126.740000   

                price_morning_max  price_evening_max  price_standard  \
year month day                                                         
2016 1     1                48.03              60.08         45.8275   
           2                56.05             118.49         50.2275   
           3                56.01             118.47         50.6050   
           4                80.02             177.66         80.0200   
           5               148.40             181.40         80.0500   
...                           ...                ...             ...   
2021 9     26               39.96              57.26         33.7800   
           27              204.31             304.75         80.0700   
           28              200.03             303.65         80.0150   
           29              180.10             304.25         80.0200   
           30              180.03             300.09         77.0500   

                     date  
year month day             
2016 1     1   2016-01-01  
           2   2016-01-02  
           3   2016-01-03  
           4   2016-01-04  
           5   2016-01-05  
...                   ...  
2021 9     26  2021-09-26  
           27  2021-09-27  
           28  2021-09-28  
           29  2021-09-29  
           30  2021-09-30  

[2100 rows x 13 columns]

In [34]:
hourly['cat']='off'

if (pref=='dam'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==6),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=7) & (hourly['hour']<=9),'cat']='morning'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=10) & (hourly['hour']<=16),'cat']='standard'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & ((hourly['month']<=5) | (hourly['month']>=9)),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']<=3),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==19),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==21),'cat']='standard'

    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==19),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='standard'

if (pref=='fpm'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=21),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'

if (pref=='fpw'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==5),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=8),'cat']='morning'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=9) & (hourly['hour']<=17),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='evening'
    
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'
    
hourly

,price,volume,hour,year,month,day,day_of_week,value,cat
date,,,,,,,,,
2016-01-01,40.10,477.5,0,2016,1,1,4,19147.750,off
2016-01-01,40.08,486.6,1,2016,1,1,4,19502.928,off
2016-01-01,40.06,495.6,2,2016,1,1,4,19853.736,off
2016-01-01,40.08,496.6,3,2016,1,1,4,19903.728,off
2016-01-01,40.07,502.9,4,2016,1,1,4,20151.203,off
...,...,...,...,...,...,...,...,...,...
2021-09-30,300.08,42.0,19,2021,9,30,3,12603.360,evening
2021-09-30,253.30,128.0,20,2021,9,30,3,32422.400,standard
2021-09-30,77.08,161.0,21,2021,9,30,3,12409.880,standard


In [35]:
daily['off_price_mean']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['standard_price_mean']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['morning_price_mean']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['evening_price_mean']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['off_hour_count']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['standard_hour_count']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['morning_hour_count']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['evening_hour_count']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()

daily['off_volume']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['standard_volume']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['morning_volume']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['evening_volume']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()

daily['off_value']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['standard_value']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['morning_value']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['evening_value']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()

daily

volume       value  price_weight_av  price_max  price_min  \
year month day                                                              
2016 1     1    7873.8  336882.508        42.785251      60.08      35.09   
           2    7319.5  345982.021        47.268532     118.49      40.03   
           3    7198.7  351712.733        48.857812     118.47      44.06   
           4    4457.0  225938.810        50.693024     177.66      46.70   
           5    3750.4  179068.882        47.746609     181.40      42.06   
...                ...         ...              ...        ...        ...   
2021 9     26   3809.0  141505.170        37.150215      57.26      32.06   
           27   3148.6  320320.984       101.734417     304.75      55.06   
           28   3059.4  299921.978        98.032940     303.65      50.09   
           29   2496.3  280136.224       112.220576     304.25      53.13   
           30   2534.8  289501.568       114.210813     300.09      50.09   

                price_mean  power_max  power_min  power_mean  \
year month day                                                 
2016 1     1     45.982500      502.9       53.0  328.075000   
           2     54.114167      486.0       40.0  304.979167   
           3     54.959167      486.0       40.0  299.945833   
           4     80.164583      486.0        0.0  185.708333   
           5     84.503333      436.0        0.0  156.266667   
...                    ...        ...        ...         ...   
2021 9     26    38.151250      212.0       56.0  158.708333   
           27   122.924167      216.5        5.0  131.191667   
           28   121.011667      190.5       27.0  127.475000   
           29   132.382000      189.4       15.0  124.815000   
           30   130.391500      193.0       41.1  126.740000   

                price_morning_max  ...  morning_hour_count  \
year month day                     ...                       
2016 1     1                48.03  ...                 3.0   
           2                56.05  ...                 NaN   
           3                56.01  ...                 NaN   
           4                80.02  ...                 3.0   
           5               148.40  ...                 3.0   
...                           ...  ...                 ...   
2021 9     26               39.96  ...                 NaN   
           27              204.31  ...                 3.0   
           28              200.03  ...                 3.0   
           29              180.10  ...                 3.0   
           30              180.03  ...                 3.0   

                evening_hour_count off_volume  standard_volume  \
year month day                                                   
2016 1     1                   2.0     3851.5           2920.3   
           2                   NaN     5969.5           1350.0   
           3                   NaN     7198.7              NaN   
           4                   2.0     3828.0            464.0   
           5                   2.0     3230.0            368.4   
...                            ...        ...              ...   
2021 9     26                  NaN     3809.0              NaN   
           27                  2.0     1290.2           1378.0   
           28                  2.0     1431.6           1188.0   
           29                  2.0      680.7           1375.0   
           30                  2.0      711.7           1364.0   

                morning_volume  evening_volume   off_value  standard_value  \
year month day                                                               
2016 1     1             993.0           109.0  152306.418      131289.730   
           2               NaN             NaN  271567.021       74415.000   
           3               NaN             NaN  351712.733             NaN   
           4             109.0            56.0  179880.680       32852.590   
           5             109.0            43.0 

In [36]:
daily.to_csv(out + pref + '_daily.csv')

In [37]:
hourly.to_csv(out + pref + '_hourly.csv')

In [38]:
hourly_flat=hourly.reset_index().set_index(['date'])
hourly_slim=hourly_flat[['hour','price','volume','cat']]
hourly_slim

,hour,price,volume,cat
date,,,,
2016-01-01,0,40.10,477.5,off
2016-01-01,1,40.08,486.6,off
2016-01-01,2,40.06,495.6,off
2016-01-01,3,40.08,496.6,off
2016-01-01,4,40.07,502.9,off
...,...,...,...,...
2021-09-30,19,300.08,42.0,evening
2021-09-30,20,253.30,128.0,standard
2021-09-30,21,77.08,161.0,standard


In [39]:
hourly_slim.to_csv(out + pref + '_hourly_slim.csv')

In [40]:
monthly=daily.groupby(['year','month']).mean().reset_index(['month','year'])
monthly=monthly.drop(['morning_volume','off_volume','standard_volume','evening_volume','price_weight_av'],1)
monthly.rename(columns={'value':'value_mean','volume':'volume_mean','off_hour_count':'off_hour_mean','standard_hour_count':'standard_hour_mean','morning_hour_count':'morning_hour_mean','evening_hour_count':'evening_hour_mean'}, inplace=True)
monthly[['price_max','standard_price_max','off_price_max','morning_price_max','evening_price_max']]=daily.groupby(['year','month']).max().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_min','standard_price_min','off_price_min','morning_price_min','evening_price_min']]=daily.groupby(['year','month']).min().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_stdev','standard_price_stdev','off_price_stdev','morning_price_stdev','evening_price_stdev']]=daily.groupby(['year','month']).std().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]

monthly[['volume','morning_volume','evening_volume','standard_volume','off_volume']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['volume','morning_volume','evening_volume','standard_volume','off_volume']]
monthly[['value','morning_value','evening_value','standard_value','off_value']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['value','morning_value','evening_value','standard_value','off_value']]

monthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]

monthly['price_weight_mean']=monthly['value'] / monthly['volume']
monthly['off_price_weight_mean']=monthly['off_value'] / monthly['off_volume']
monthly['standard_price_weight_mean']=monthly['standard_value'] / monthly['standard_volume']
monthly['morning_price_weight_mean']=monthly['morning_value'] / monthly['morning_volume']
monthly['evening_price_weight_mean']=monthly['evening_value'] / monthly['evening_volume']


monthly['off_power']=monthly['off_volume'] / monthly['off_hour_count']
monthly['morning_power']=monthly['morning_volume'] / monthly['morning_hour_count']
monthly['standard_power']=monthly['standard_volume'] / monthly['standard_hour_count']
monthly['evening_power']=monthly['evening_volume'] / monthly['evening_hour_count']

monthly

,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2016,1,3785.351613,211930.289323,106.684000,45.982500,82.458788,362.651613,17.606452,159.959113,...,42.0,55.986949,50.402420,81.002858,77.850006,137.720998,273.188920,33.409524,56.583459,37.350000
1,2016,2,3871.755172,261082.316897,123.798500,75.023333,91.797506,367.334483,23.231034,161.993793,...,42.0,67.432548,56.605271,106.547628,129.026815,156.801247,274.654573,12.280952,75.799614,42.580952
2,2016,3,2904.254839,187713.287290,108.651500,60.115833,88.953989,285.867742,18.232258,122.263199,...,46.0,64.633890,55.295628,98.717668,116.099471,107.341031,212.547024,15.502899,56.748043,34.786957
3,2016,4,1921.116667,118965.331100,114.641667,42.853500,85.579278,183.970000,7.236667,80.344000,...,42.0,61.925094,57.116312,104.192200,104.410448,139.262082,152.162463,7.793651,19.055263,4.438095
4,2016,5,1597.551613,89924.522194,105.676500,28.659583,79.317876,165.183871,4.661290,68.442769,...,44.0,56.288962,49.684509,106.019186,95.368696,140.331232,124.132670,10.831818,18.262963,4.170455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,2021,5,2501.206452,168242.250258,113.129583,34.269167,84.021403,162.409677,48.145161,106.544812,...,42.0,67.264440,37.138276,90.791796,145.617597,285.751219,130.365097,81.076190,87.254511,51.383333
65,2021,6,2408.913333,258848.577167,145.522500,52.754167,121.474997,183.256667,33.226667,101.923778,...,66.0,107.454500,56.446636,131.411501,197.932285,289.100821,114.970732,124.809091,90.660317,52.624242
66,2021,7,3174.064516,328619.638677,148.904500,35.070000,113.573901,213.532258,59.832258,134.900081,...,66.0,103.532753,50.103428,128.585369,194.682037,289.336072,147.687059,143.656061,128.534615,80.031818
67,2021,8,3965.867742,377464.605516,138.765417,35.761667,106.275995,271.070968,89.264516,168.561694,...,66.0,95.178314,45.770251,120.966602,189.848214,282.911328,193.331322,162.575758,150.134524,107.556061


In [41]:
if (pref=='dam'):
    monthly['evening_days']=daily.loc[daily['evening_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['morning_days']=daily.loc[daily['morning_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['days']=daily.groupby(['year','month']).count().reset_index(['month','year'])['volume']

monthly

,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power,evening_days,morning_days,days
0,2016,1,3785.351613,211930.289323,106.684000,45.982500,82.458788,362.651613,17.606452,159.959113,...,81.002858,77.850006,137.720998,273.188920,33.409524,56.583459,37.350000,21,21,31
1,2016,2,3871.755172,261082.316897,123.798500,75.023333,91.797506,367.334483,23.231034,161.993793,...,106.547628,129.026815,156.801247,274.654573,12.280952,75.799614,42.580952,21,21,29
2,2016,3,2904.254839,187713.287290,108.651500,60.115833,88.953989,285.867742,18.232258,122.263199,...,98.717668,116.099471,107.341031,212.547024,15.502899,56.748043,34.786957,23,23,31
3,2016,4,1921.116667,118965.331100,114.641667,42.853500,85.579278,183.970000,7.236667,80.344000,...,104.192200,104.410448,139.262082,152.162463,7.793651,19.055263,4.438095,21,21,30
4,2016,5,1597.551613,89924.522194,105.676500,28.659583,79.317876,165.183871,4.661290,68.442769,...,106.019186,95.368696,140.331232,124.132670,10.831818,18.262963,4.170455,22,22,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,2021,5,2501.206452,168242.250258,113.129583,34.269167,84.021403,162.409677,48.145161,106.544812,...,90.791796,145.617597,285.751219,130.365097,81.076190,87.254511,51.383333,21,21,31
65,2021,6,2408.913333,258848.577167,145.522500,52.754167,121.474997,183.256667,33.226667,101.923778,...,131.411501,197.932285,289.100821,114.970732,124.809091,90.660317,52.624242,22,22,30
66,2021,7,3174.064516,328619.638677,148.904500,35.070000,113.573901,213.532258,59.832258,134.900081,...,128.585369,194.682037,289.336072,147.687059,143.656061,128.534615,80.031818,22,22,31
67,2021,8,3965.867742,377464.605516,138.765417,35.761667,106.275995,271.070968,89.264516,168.561694,...,120.966602,189.848214,282.911328,193.331322,162.575758,150.134524,107.556061,22,22,31


In [42]:
monthly=monthly.set_index(['year','month'])

In [43]:
monthly['off_price_diff']=monthly['off_price_mean']-monthly['off_price_mean'].shift(1)
monthly['morning_price_diff']=monthly['morning_price_mean']-monthly['morning_price_mean'].shift(1)
monthly['standard_price_diff']=monthly['standard_price_mean']-monthly['standard_price_mean'].shift(1)
monthly['evening_price_diff']=monthly['evening_price_mean']-monthly['evening_price_mean'].shift(1)

monthly['off_price_log_diff']=np.log10(monthly['off_price_mean'])-np.log10(monthly['off_price_mean']).shift(1)
monthly['morning_price_log_diff']=np.log10(monthly['morning_price_mean'])-np.log10(monthly['morning_price_mean']).shift(1)
monthly['standard_price_log_diff']=np.log10(monthly['standard_price_mean'])-np.log10(monthly['standard_price_mean']).shift(1)
monthly['evening_price_log_diff']=np.log10(monthly['evening_price_mean'])-np.log10(monthly['evening_price_mean']).shift(1)

In [44]:
monthly.to_csv(out + pref + '_monthly.csv')

In [45]:
monthly_filt=monthly #.loc[monthly['year']<2020]
daily_filt=daily.reset_index() #.loc[daily.reset_index()['year']<2020]

In [46]:
calmonthly=daily_filt.groupby(['month']).mean().reset_index()
calmonthly=calmonthly.drop(['year','day','volume','standard_volume','off_volume','morning_volume','evening_volume', 'off_hour_count', 'morning_hour_count', 'standard_hour_count','evening_hour_count'],1)
calmonthly.rename(columns={'value':'value_mean','volume':'volume_mean','price_weight_av': 'price_weight_mean'}, inplace=True)

calmonthly[['price_median', 'off_price_median','standard_price_median', 'morning_price_median', 'evening_price_median']]=daily_filt.groupby(['month']).median().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]
calmonthly[['price_stdev', 'off_price_stdev','standard_price_stdev', 'morning_price_stdev', 'evening_price_stdev']]=daily_filt.groupby(['month']).std().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]

calmonthly[['volume', 'off_volume','standard_volume', 'morning_volume', 'evening_volume']]=monthly_filt.groupby(['month']).mean().reset_index()[['volume','off_volume','standard_volume', 'morning_volume', 'evening_volume']]
calmonthly[['value', 'off_value','standard_value', 'morning_value', 'evening_value']]=monthly_filt.groupby(['month']).mean().reset_index()[['value','off_value','standard_value', 'morning_value', 'evening_value']]
calmonthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]
calmonthly[['off_power','standard_power', 'morning_power', 'evening_power']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_power','standard_power', 'morning_power', 'evening_power']]

calmonthly

,month,value_mean,price_weight_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,price_morning_max,...,evening_volume,value,off_hour_count,standard_hour_count,morning_hour_count,evening_hour_count,off_power,standard_power,morning_power,evening_power
0,1,185600.053215,51.791707,129.461989,36.609624,62.138019,281.433333,43.951075,156.382088,79.971344,...,2721.516667,5.753602e+06,347.000000,274.166667,66.5,44.333333,210.699466,115.574718,103.699394,60.955111
1,2,180268.128012,58.427649,145.418706,35.236765,66.698482,248.718824,37.573529,135.031676,93.969412,...,2639.716667,5.107597e+06,326.833333,251.000000,60.5,40.333333,167.617240,111.956236,105.720714,65.638075
2,3,132341.096522,53.445720,126.591237,31.432151,60.910259,191.055376,42.351613,105.442899,87.328548,...,2915.566667,4.102574e+06,344.333333,276.000000,67.0,44.666667,122.936379,93.665556,93.246158,65.405442
3,4,142213.911511,49.068720,120.105222,28.292722,56.505089,200.642778,50.254444,123.747944,82.026889,...,3586.533333,4.266417e+06,340.333333,265.000000,64.0,42.666667,139.339475,110.861947,123.269030,84.363088
4,5,126268.350199,49.917404,140.868387,29.335484,60.296361,193.709677,40.886022,106.046832,82.781129,...,2455.500000,3.914319e+06,347.000000,274.166667,66.5,44.333333,136.338227,79.504330,90.868968,55.137571
5,6,214057.784206,66.253087,161.234833,36.383278,77.417112,269.190556,46.628889,139.203921,114.329500,...,5597.766667,6.421734e+06,333.333333,249.666667,64.5,64.500000,177.778448,103.403080,125.502374,87.643241
6,7,271957.968382,69.272739,168.801183,38.594355,80.492093,304.971505,58.989247,166.295968,125.498548,...,6480.866667,8.430697e+06,344.000000,256.000000,66.0,66.000000,214.353014,126.650901,135.118492,97.062966
7,8,300164.724790,66.900762,165.620323,36.001828,78.735378,351.583333,71.529032,192.473002,124.118925,...,7656.066667,9.305106e+06,340.000000,258.000000,67.0,67.000000,250.967090,149.177571,148.881696,113.629227
8,9,294415.718811,57.949315,140.910556,33.358833,67.969095,378.627222,92.178889,220.605463,96.025944,...,5130.800000,8.832472e+06,340.333333,265.000000,64.0,42.666667,282.095454,171.800431,168.346396,123.047152
9,10,309061.279226,50.411789,115.442839,31.392516,57.464071,431.950323,102.941290,266.851339,77.312258,...,4317.460000,9.580900e+06,346.000000,275.000000,66.6,44.400000,341.262041,219.603449,182.784896,96.877648


In [47]:
calmonthly.to_csv(out + pref + '_calmonthly.csv')

In [75]:
calmonthly_hours=hourly.groupby(['month','hour']).mean()
calmonthly_hours=calmonthly_hours.drop(['year','day','day_of_week'],1)
calmonthly_hours

price     volume        value
month hour                                    
1     0      38.381087  11.568478   509.468380
      1      38.381087  11.568478   509.468380
      2      38.381087  11.568478   509.468380
      3      38.381087  11.568478   509.468380
      4      38.316935  11.626613   513.657411
...                ...        ...          ...
12    19    121.692500  14.888710   853.341532
      20     65.951855  25.118548  1396.648806
      21     65.951855  25.118548  1396.648806
      22     35.736452  46.086290  1868.078177
      23     35.736452  46.086290  1868.078177

[288 rows x 3 columns]

In [76]:
calmonthly_hours.to_csv(out + pref + '_calmonthly_hours.csv')

In [77]:
from statsmodels.tsa.stattools import pacf

In [78]:
pacfs=pd.DataFrame(data={'standard': pacf(monthly.reset_index()['standard_price_mean'], nlags=14)})
pacfs['evening']= pacf(monthly.reset_index()['evening_price_mean'], nlags=14)
pacfs['morning']= pacf(monthly.reset_index()['morning_price_mean'], nlags=14)
pacfs['off']= pacf(monthly.reset_index()['off_price_mean'], nlags=14)
pacfs

,standard,evening,morning,off
0,1.000000,1.000000,1.000000,1.000000
1,0.675201,0.703333,0.708057,0.807327
2,-0.108083,-0.210052,-0.204584,-0.384366
3,0.119619,0.287313,0.268888,0.142507
4,0.201097,0.162768,0.188562,0.154830
5,0.073670,0.163157,0.125566,0.148409
6,0.124138,0.091377,0.140205,-0.046410
7,0.000326,-0.153420,-0.173091,-0.131391
8,-0.094432,0.086606,0.074670,0.002911
9,-0.138679,-0.289313,-0.277447,-0.191344


In [79]:
pacfs.to_csv(out + pref + '_pacfs.csv')